In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np

In [15]:
df = pd.read_csv('GISP20002019.csv')
print(df.head())

df.iloc[:, 5:len(df.columns)] = df.iloc[:, 5:len(df.columns)].apply(pd.to_numeric, errors='coerce')
## errors = coerce converts missing data to NaN
print(df.head())

## so e.g. in Alabama, in 2000, there were 143 MSW who were sampled, only 10 of which were susceptible to all drugs etc ect 

   Obs CLINIC  YEAR         GENDERSP  TOTAL Susceptible TetI PenI PenI_TetI  \
0    1    ALB  2000              MSW    143          10    2   36        68   
1    2    ALB  2000              MSM      9           .    .    1         5   
2    3    ALB  2000             MSMW      5           .    .    2         3   
3    4    ALB  2000  Oth/Unk/Missing      4           .    1    .         1   
4    5    ALB  2001              MSW    127          12    3   40        57   

  TetR  ... AziRS_TetR_CipI_PenI CipR CipR_TetR AziRS_CipI_PenI_TetI  \
0    2  ...                    .    .         .                    .   
1    .  ...                    .    .         .                    .   
2    .  ...                    .    .         .                    .   
3    .  ...                    .    .         .                    .   
4    1  ...                    .    .         .                    .   

  CfxRS_CipR_TetR_PenI AziRS_TetR AziRS_CipR_PenR_TetI  \
0                    .          . 

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_45588/2212419462.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 5:len(df.columns)] = df.iloc[:, 5:len(df.columns)].apply(pd.to_numeric, errors='coerce')


In [16]:
## get only CIP data 
drugsDict = {}
drugsDict["CIP"] = list(df.columns)

## only get columns where CipR exists in title 
for variable in range(len(list(df.columns))):
    if "CipR" not in drugsDict["CIP"][variable]: ## so this is saying that if CipR is not in the variable (comparing susbsections of strings?)
        drugsDict["CIP"][variable] = None

drugsDict["CIP"] = list(filter(None, drugsDict["CIP"]))


CIP_colums = list(df.columns[0:6]) + list(filter(None, drugsDict["CIP"]))
print(CIP_colums)

CIP_data_full = df[CIP_colums]
#print(df[[CIP_colums]])
CIP_data_full.head()
CIP_data_full.tail()


['Obs', 'CLINIC', 'YEAR', 'GENDERSP', 'TOTAL', 'Susceptible', 'CipR_PenR_TetR', 'CipR_PenI_TetI', 'CipR_TetR_PenI', 'CipR_PenI', 'CipR_PenR_TetI', 'AziRS_CipR_PenR_TetR', 'CipR_PenR', 'AziRS_CipR_TetR_PenI', 'CfxRS_CipR_PenR_TetR', 'AziRS_CipR_PenI_TetI', 'CipR_TetI', 'CipR', 'CipR_TetR', 'CfxRS_CipR_TetR_PenI', 'AziRS_CipR_PenR_TetI', 'CroRS_CfxRS_CipR_PenR_TetR']


,Obs,CLINIC,YEAR,GENDERSP,TOTAL,Susceptible,CipR_PenR_TetR,CipR_PenI_TetI,CipR_TetR_PenI,CipR_PenI,...,CipR_PenR,AziRS_CipR_TetR_PenI,CfxRS_CipR_PenR_TetR,AziRS_CipR_PenI_TetI,CipR_TetI,CipR,CipR_TetR,CfxRS_CipR_TetR_PenI,AziRS_CipR_PenR_TetI,CroRS_CfxRS_CipR_PenR_TetR
1965,1966,WDC,2018,MSM,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1966,1967,WDC,2019,MSW,45,3.0,1.0,5.0,2.0,1.0,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN
1967,1968,WDC,2019,MSM,25,1.0,NaN,5.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968,1969,WDC,2019,MSMW,6,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969,1970,WDC,2019,Oth/Unk/Missing,1,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
## now need to start disaggregating... 
print(CIP_data_full["TOTAL"].sum())
## so 112,487 in total had some resistance to CIP. Seems like a lot? maybe not though
## means that I want a DF with 112,487 

## different levels for the two variables of interest - sexual behaviours and geographic area
## sexual characteristics
GenderSP = list(CIP_data_full["GENDERSP"].unique())
Location = list(CIP_data_full["CLINIC"].unique())
Year = list(CIP_data_full["YEAR"].unique())
#print(len(GenderSP)*len(Location)*len(Year)) # 3360 possible identifiers

## what are we interested in? in the first case, geography, sexual activity, year, and CIP (for the moment just bundle everything under "CIPR")
#
### so will abbreviate dataframe so that it is "YEAR" "LOCATION" "GENDERSP" "CIPR" (which will be a binary variable)
CIP_data_full = CIP_data_full.replace(np.nan, 0) ##otherwise cannot seem to sum? 
print(CIP_data_full.columns)

### need to redefine SUSCEPTIBLE so it is just susceptible to CIPR, not all other drugs
CIP_data_full["Susceptible"] =  CIP_data_full["TOTAL"] - CIP_data_full[['CipR_PenR_TetR', 'CipR_PenI_TetI', 'CipR_TetR_PenI', 'CipR_PenI',
       'CipR_PenR_TetI', 'AziRS_CipR_PenR_TetR', 'CipR_PenR',
       'AziRS_CipR_TetR_PenI', 'CfxRS_CipR_PenR_TetR', 'AziRS_CipR_PenI_TetI',
       'CipR_TetI', 'CipR', 'CipR_TetR', 'CfxRS_CipR_TetR_PenI',
       'AziRS_CipR_PenR_TetI', 'CroRS_CfxRS_CipR_PenR_TetR']].sum(axis = 1) # Which is (TOTAL SAMPLES) - (SUM OF ALL THE ONES THAT SHOWED CIPRO RESISTANCE)


## So now to get susceptible to CIPR 


CIP_data_abb = CIP_data_full[["CLINIC", "YEAR", "GENDERSP", "TOTAL", "Susceptible"]]
#print(CIP_data_abb.tail())

112487
Index(['Obs', 'CLINIC', 'YEAR', 'GENDERSP', 'TOTAL', 'Susceptible',
       'CipR_PenR_TetR', 'CipR_PenI_TetI', 'CipR_TetR_PenI', 'CipR_PenI',
       'CipR_PenR_TetI', 'AziRS_CipR_PenR_TetR', 'CipR_PenR',
       'AziRS_CipR_TetR_PenI', 'CfxRS_CipR_PenR_TetR', 'AziRS_CipR_PenI_TetI',
       'CipR_TetI', 'CipR', 'CipR_TetR', 'CfxRS_CipR_TetR_PenI',
       'AziRS_CipR_PenR_TetI', 'CroRS_CfxRS_CipR_PenR_TetR'],
      dtype='object')


In [20]:
print(CIP_data_abb["TOTAL"] - CIP_data_abb["Susceptible"])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
1965     0.0
1966    14.0
1967    13.0
1968     4.0
1969     1.0
Length: 1970, dtype: float64


In [7]:
###

df=CIP_data_abb.reindex(CIP_data_abb.index.repeat(CIP_data_abb.TOTAL))

df=df.assign(TOTAL=1) ## can use this to count. So when the total in rows is equal to the number of Susceptible for that year/gender ect., , then the value of susceptible changes from 1 to 0.
df.head()

df.CIPR = pd.to_numeric(df.Susceptible)
df.Susceptible=np.where(df.groupby(level=0).cumcount()<df.Susceptible,1,0) # basically once the cumulative total reaches == the value for susceptible in that row, switch from 0 to 1
#
print(df.shape)
df.columns
### make a dataframe 
#CIP_disag = pd.df()
print(df.head(15))

CIP_disaggregated = df[["CLINIC", "YEAR", "GENDERSP", "Susceptible"]]
CIP_disaggregated.to_csv("CIP_Resistant_disagregated.csv")




(112487, 5)
  CLINIC  YEAR GENDERSP  TOTAL  Susceptible
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1
0    ALB  2000      MSW      1            1


/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_45588/271577498.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.CIPR = pd.to_numeric(df.Susceptible)


In [11]:
### quick check - 2019 
CIP_data_full["TOTAL"]


0       143
1         9
2         5
3         4
4       127
       ... 
1965      1
1966     45
1967     25
1968      6
1969      1
Name: TOTAL, Length: 1970, dtype: int64

In [8]:
CIP_data_2019 = CIP_disaggregated.loc[CIP_disaggregated['YEAR'] == 2019]

print(1 - CIP_data_2019['Susceptible'].sum()/len(CIP_data_2019['Susceptible']))


0.3542199488491049


In [26]:
CIP_data_2019 = CIP_disaggregated.loc[CIP_disaggregated['YEAR'] == 2019]
CIP_data_2019.loc[CIP_data_2019['GENDERSP'] == "MSM"]
print(1 - CIP_data_2019['Susceptible'].sum()/len(CIP_data_2019['Susceptible']))

0.3542199488491049
